### 드라이브 마운트

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 패키지 설치

In [19]:
!pip install langchain langchain_google_genai

In [20]:
from pprint import pprint
from typing import Dict, List

from langchain.chains import LLMChain, SequentialChain
#from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key

In [26]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


### Prompt chain 준비
* 서비스할 내용의 프롬프트 체인을 준비합니다.
* 각 프롬프트 체인을 미리 준비해 놓고, 템플릿으로 사용합니다.

In [27]:
P_PATH = "/content/drive/MyDrive/dataset/Novel_generation/multi_prompt"
IDEA_P = os.path.join(P_PATH, "extract_idea.txt")
OUTLINE_P = os.path.join(P_PATH, "write_outline.txt")
PLOT_P = os.path.join(P_PATH, "write_plot.txt")
CHAPTER_P = os.path.join(P_PATH, "write_chapter.txt")

### Prompt chain 구현
* `SequentialChain`을 이용해서 여러개의 chain을 연속적으로 구현할 수 있습니다.

In [28]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]]
    text: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path),
        ),
        output_key=output_key,
        verbose=True,
    )


def generate_novel(req: UserRequest) -> Dict[str, str]:
    writer_llm = ChatGoogleGenerativeAI(temperature=0.3, max_tokens=500, model="gemini-pro")

    # 아이디어 뽑기 체인 생성
    novel_idea_chain = create_chain(writer_llm,IDEA_P, "novel_idea")

    # 아웃라인 작성 체인 생성
    novel_outline_chain = create_chain(
        writer_llm, OUTLINE_P, "novel_outline"
    )

    # 플롯 작성 체인 생성
    novel_plot_chain = create_chain(writer_llm, PLOT_P, "novel_plot")

    # 챕터 작성 체인 생성
    novel_chapter_chain = create_chain(writer_llm, CHAPTER_P, "output")

    preprocess_chain = SequentialChain(
        chains=[
            novel_idea_chain,
            novel_outline_chain,
            novel_plot_chain,
        ],
        input_variables=["genre", "characters", "text"],
        output_variables=["novel_idea", "novel_outline", "novel_plot"],
        verbose=True,
    )

    context = req.dict()
    context = preprocess_chain(user_data)

    context["novel_chapter"] = []
    for chapter_number in range(1, 4):
        context["chapter_number"] = chapter_number
        context = novel_chapter_chain(context)
        context["novel_chapter"].append(context["output"])

    contents = "\n\n".join(context["novel_chapter"])
    return {"results": contents}

### User prompt 작성
* User가 직접 작성하는 프롬프트를 작성합니다.

In [29]:
user_data = {
    "genre": "판타지",
    "characters": [
        {
            "name": "김철수",
            "role": "주인공"
        },
        {
            "name": "이영희",
            "role": "조연"
        }
    ],
    "text": "날씨가 추워지고 있습니다."
}


* User Prompt를 입력합니다.

In [30]:
request_instance = UserRequest(**user_data)

### Text Generation

In [31]:
generate_novel(request_instance)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[등장 인물] 과 [참고 텍스트] 를 소재로 새롭고 흥미진진한 판타지 소설 아이디어를 한 문단으로 작성해줘

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '김철수', 'role': '주인공'}, {'name': '이영희', 'role': '조연'}]

[참고 텍스트]
날씨가 추워지고 있습니다.

[아이디어]
추운 겨울날, 김철수와 이영희는 숲속을 걷고 있었다. 눈이 내리고 있었고, 나무들은 얼어붙어 있었다. 갑자기, 그들은 이상한 소리를 들었다. 그들은 소리를 따라가다가 얼음으로 덮인 동굴에 도착했다. 그들은 동굴 안으로 들어가 보았고, 그곳에서 이상한 생물을 발견했다. 그 생물은 흰색의 털을 가지고 있었고, 눈은 푸른색이었다. 그들은 그 생물에게 다가가 물었다. "당신은 누구인가요?" 그 생물은 대답했다. "나는 눈의 여왕입니다." 눈의 여왕은 그들에게 말했다. "나는 이 숲을 지키는 자입니다. 그리고 지금, 나는 당신들에게 한 가지 임무를 주고 싶습니다." 그들은 눈의 여왕에게 물었다. "무슨 임무입니까?" 눈의 여왕은 대답했다. "나는 당신들에게 숲의 마법을 지키는 열쇠를 찾아오기를 바랍니다. 그 열쇠는 숲의 어딘가에 숨겨져 있습니다. 그리고 당신들은 그 열쇠를 찾아와야 합니다." 그들은 눈의 여왕에게 물었다. "그 열쇠를 찾으면 어떻게 되나요?" 눈의 여왕은 대답했다. "그 열쇠를 찾으면, 당신들은 숲의 마법을 지킬 수 있을 것입니다. 그리고 당신들은 

{'results': '**Chapter 1: 숲속의 이상한 소리**\n\n추운 겨울날, 김철수와 이영희는 숲속을 걷고 있었다. 눈이 내리고 있었고, 나무들은 얼어붙어 있었다. 그들은 겨울방학을 이용해 캠핑을 하기로 했다. 둘은 어릴 때부터 절친한 친구였고, 항상 함께 놀곤 했다.\n\n그들은 숲속을 걸으며 이야기를 나누고 있었다. 그들은 학교에서 있었던 일과, 앞으로의 계획에 대해 이야기했다. 그들은 숲속을 걸으며 즐거운 시간을 보냈다.\n\n갑자기, 그들은 이상한 소리를 들었다. 그것은 숲속에서 나는 소리였고, 그들은 그 소리가 무엇인지 알 수 없었다. 그들은 소리를 따라가기로 했다.\n\n그들은 소리를 따라가다가 얼음으로 덮인 동굴에 도착했다. 동굴은 어두컴컴했고, 그들은 안으로 들어가기가 두려웠다. 그러나 그들은 용기를 내어 동굴 안으로 들어갔다.\n\n그들은 동굴 안으로 들어가 보았고, 그곳에서 이상한 생물을 발견했다. 그 생물은 흰색의 털을 가지고 있었고, 눈은 푸른색이었다. 그들은 그 생물에게 다가가 물었다.\n\n"당신은 누구인가요?"\n\n그 생물은 대답했다.\n\n"나는 눈의 여왕입니다."\n\n눈의 여왕은 그들에게 말했다.\n\n"나는 이 숲을 지키는 자입니다. 그리고 지금, 나는 당신들에게 한 가지 임무를 주고 싶습니다."\n\n그들은 눈의 여왕에게 물었다.\n\n"무슨 임무입니까?"\n\n눈의 여왕은 대답했다.\n\n"나는 당신들에게 숲의 마법을 지키는 열쇠를 찾아오기를 바랍니다. 그 열쇠는 숲의 어딘가에 숨겨져 있습니다. 그리고 당신들은 그 열쇠를 찾아와야 합니다."\n\n그들은 눈의 여왕에게 물었다.\n\n"그 열쇠를 찾으면 어떻게 되나요?"\n\n눈의 여왕은 대답했다.\n\n"그 열쇠를 찾으면, 당신들은 숲의 마법을 지킬 수 있을 것입니다. 그리고 당신들은 숲의 평화를 지킬 수 있을 것입니다."\n\n그들은 눈의 여왕에게 말했다.\n\n"우리는 그 열쇠를 찾아올 것입니다."\n\n그리고 그들은 숲을 뒤지기 시작했다.\n\n**Cha